In [1]:
import Pkg
Pkg.activate(".")
using DataFrames, Feather

 Activating environment at `~/Documents/Julia/PkgEvalAnalysis/Project.toml`
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling Feather [becb17da-46f6-5d3c-ad1b-1c5fe96bc73c]
└ @ Base loading.jl:1278


In [108]:
@enum FailureType begin
    # General ones
    MacroTools
    Unknown
    CanonicalTraits
    ApproxError
    AstroTime
    TestStdlib
    JuliaInterpreter
    PairwiseListMatrices
    PDDL
    QuickTypes
    TensorKit
end

In [95]:
if !isdir("data")
    run(`tar -xvf data.tar.gz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

In [96]:
package_results = join(primary, against,
     on=:uuid, kind=:left, makeunique=true, indicator=:source);

In [97]:
changed_tests = filter(test->test.source == "both" &&
                 test.status != test.status_1, package_results)
fails = changed_tests[changed_tests[!, :status] .== ":fail", :];
sort!(fails , :name);
insert!(fails, size(fails, 2), fill(Unknown, size(fails, 1)), :why);

In [98]:
query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [109]:
# Generic ones
update_reason!(fails, "@ MacroTools ~/.julia/packages/MacroTools/X77lQ/src/utils.jl:322", MacroTools)
update_reason!(fails, "@ MacroTools ~/.julia/packages/MacroTools/X77lQ/src/utils.jl:327", MacroTools)
update_reason!(fails, "@ MacroTools ~/.julia/packages/MacroTools/X77lQ/src/utils.jl:334", MacroTools)
update_reason!(fails, "@ ~/.julia/packages/MacroTools/X77lQ/src/utils.jl:44", MacroTools)
update_reason!(fails, "@ Zygote ~/.julia/packages/Zygote/1GXzF/src/lib/grad.jl:7", MacroTools)
update_reason!(fails, "@ ~/.julia/packages/MacroTools/X77lQ/src/match/macro.jl:72", MacroTools)

update_reason!(fails, "@ CanonicalTraits ~/.julia/packages/CanonicalTraits/jEjaE/src/Typeclasses.jl:42", CanonicalTraits)
update_reason!(fails, "@ AstroTime ~/.julia/packages/AstroTime/ArvEM/src/AstroTime.jl:160", AstroTime)
update_reason!(fails, "@ /workspace/srcdir/usr/share/julia/stdlib/v1.6/Test/src/Test.jl:1102", TestStdlib)
update_reason!(fails, "@ ~/.julia/packages/JuliaInterpreter/UwKTd/src/breakpoints.jl:416", JuliaInterpreter)
update_reason!(fails, "LoadError: syntax: invalid assignment location \"PairwiseListMatrices.t\"", PairwiseListMatrices)
update_reason!(fails, "@ PDDL ~/.julia/packages/PDDL/vLSH8/src/core.jl:84", PDDL)
update_reason!(fails, "@ QuickTypes ~/.julia/packages/QuickTypes/Uwyvo/src/QuickTypes.jl:144", QuickTypes)
update_reason!(fails, "@ ~/.julia/packages/TensorKit/CMOeD/test/sectors.jl:12", TensorKit)

update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, "Expression: ≈", ApproxError)

nothing

In [110]:
total_unknown(fails)

9

In [118]:
filter(x -> x.why == Unknown, fails).name
filter(x -> x.why == TestStdlib, fails).name

5-element Array{String,1}:
 "ConstantTime"
 "Curve25519"
 "DarkCurves"
 "DarkIntegers"
 "Jute"

In [113]:
z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    push!(z, (n, i))
end
sort!(z; rev=true)

11-element Array{Any,1}:
 (223, MacroTools)
 (9, Unknown)
 (5, TestStdlib)
 (5, CanonicalTraits)
 (2, QuickTypes)
 (2, PairwiseListMatrices)
 (1, TensorKit)
 (1, PDDL)
 (1, JuliaInterpreter)
 (1, AstroTime)
 (1, ApproxError)

In [117]:
unknownfails = filter(x -> x.why == Unknown, fails)
unknownfails.undefvarerror = occursin.("UndefVarError", unknownfails.log)
unknownfails[!, [:name, :undefvarerror]]

,name,undefvarerror
,String,Bool
1,BioServices,0
2,EliminateGraphs,0
3,ITensors,0
4,LogisticOptTools,0
5,LoweredCodeUtils,1
6,NDTensors,0
7,Poltergeist,0
8,StatsKit,0
9,Theta,0
